In [192]:
import os
import sys
import shutil
import pandas
import numpy
import ipystata
import matplotlib.pyplot as plt

from ipystata.config import config_stata
config_stata('/home/kmongeon/research/stata14/stata-mp',force_batch=True)

from ipystata.config import config_syntax_higlight    
config_syntax_higlight(True)  

pandas.options.display.max_columns = 100

from IPython.display import display, HTML, Image

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.float_format = '{:.2f}'.format

In [193]:
pwd

'/home/kmongeon/research/exercise_physiology/scripts'

In [194]:
try: 
    shutil.rmtree('out')
    os.mkdir('out')
except:
    pass


In [195]:
%%stata --output dm

import delimited ./data/IzzyLongitudinalWorkingMarch2015.csv,  case(lower) clear 
order id session sequence
sort id sequence
save ./out/di, replace

import delimited ./data/torque.csv, case(lower) clear
drop if session==0
drop sequence
rename new_sequence sequence
order id session sequence
sort id  sequence
foreach var in ptiso pt60 pt240 {
replace `var' = . if `var'==-9999
}
save ./out/dt, replace

** Identify potentially data with errors
* Nota: July  2016
import delimited ./data/NegativeIDs.csv, case(lower) clear
keep nota id sequence 
keep if nota==1
save ./out/db1, replace

* Baraket: Nov 2016
import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
keep falk id sequence
keep if falk==1
save ./out/db2, replace

** additional ptiso data
import delimited ./data/Spring_2013_knee_extension.csv, case(lower) clear
save ./out/dk, replace

* merge
use ./out/di, clear
sort id sequence
merge 1:1 id sequence using ./out/dt,  nogen
merge 1:1 id sequence using ./out/db1, nogen
merge 1:1 id sequence using ./out/db2, nogen
merge 1:1 id sequence using ./out/dk 
drop if _merge==2
drop _merge
count

save ./out/dm, replace


(70 vars, 427 obs)
(note: file ./out/di.dta not found)
file ./out/di.dta saved
(14 vars, 421 obs)
(40 observations deleted)(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(note: file ./out/dt.dta not found)
file ./out/dt.dta saved
. * Nota: July  2016(32 vars, 109 obs)
(59 observations deleted)
(note: file ./out/db1.dta not found)
file ./out/db1.dta saved
. import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
(34 vars, 473 obs)
(460 observations deleted)
(note: file ./out/db2.dta not found)
file ./out/db2.dta saved
. import delimited ./data/Spring_2013_knee_extension.csv, case(lower) clear
(4 vars, 50 obs)
(note: file ./out/dk.dta not found)
file ./out/dk.dta saved
. use ./out/di, clear

    Result                           # of obs.
    -----------------------------------------
    not matched                            58
        from master                        52  
        from using           

In [196]:
%%stata --output dm

use ./out/dm

rename radius_sos rsos
rename tibial_sos tsos

rename grip_strength_best grip 
rename mat_offset_new matu

rename godin_pa godin
rename ntxcreat ntxc
rename caloricintake cint
renam calcium calc 
rename vitd vitd 
rename v48 vitd2
rename paq_total_score paq
rename totmvh mvh

rename forearm_length flength
rename thigh_length   tlength

global U id session sequence 
global Y rsos tsos
global X grip ptiso pt60 godin paq ntxc cint calc vitd vitd2 mvh
global Z matu age gender
global D nota falk

drop if matu>5
drop if id==517 & sequence==7

sort id sequence
by id: egen seq = seq()
    
global U id session sequence seq

order $U $D $Y $X $Z 

sort id sequence

gen boy = 1 if gender==0
gen girl = 1 if gender==1

replace grip = . if nota==1
replace rsos = . if nota==1
*replace tsos = . if falk==1
*replace ptiso = . if falk==1

keep id sequence session seq   ///
    nota falk rsos tsos grip ptiso ntx mvh godin paq cint matu gender
*********************************************************************************
sum *



  (all newnames==oldnames)
(7 observations deleted)
(1 observation deleted)
. global U id session sequence seq
(199 missing values generated)
(226 missing values generated)
(50 real changes made, 50 to missing)
(50 real changes made, 50 to missing)
. *replace ptiso = . if falk==1>     nota falk rsos tsos grip ptiso ntx mvh godin paq cint matu gender
. sum *

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
          id |        425    326.1976    197.5498        100        583
     session |        425    1.430588    .4957421          1          2
    sequence |        425    4.197647    1.833518          1          7
         seq |        425    1.868235    .8776861          1          4
        nota |         50           1           0          1          1
-------------+---------------------------------------------------------
        falk |         11           1           0          1  

In [197]:
dm['rsoss'] = (dm['rsos'] - dm['rsos'].mean()) / dm['rsos'].std()
dm['tsoss'] = (dm['tsos'] - dm['tsos'].mean()) / dm['tsos'].std()
dm['grips'] = (dm['grip'] - dm['grip'].mean()) / dm['grip'].std()
dm['torqs'] = (dm['ptiso'] - dm['ptiso'].mean()) / dm['ptiso'].std()

dm['ntxs'] = (dm['ntx'] - dm['ntx'].mean()) / dm['ntx'].std()
dm['mvhs'] = (dm['mvh'] - dm['mvh'].mean()) / dm['mvh'].std()
dm['gods'] = (dm['godin'] - dm['godin'].mean()) / dm['godin'].std()
dm['paqs'] = (dm['paq'] - dm['paq'].mean()) / dm['paq'].std()

dm['cints'] = (dm['cint'] - dm['cint'].mean()) / dm['cint'].std()
dm['matus'] = (dm['matu'] - dm['matu'].mean()) / dm['matu'].std()

dm.head(2)

,id,session,sequence,seq,nota,falk,rsos,tsos,grip,ptiso,godin,paq,cint,mvh,matu,gender,ntx,rsoss,tsoss,grips,torqs,ntxs,mvhs,gods,paqs,cints,matus
0,100,1,1,1,nan,nan,3828.00,3601.00,nan,113.93,37.00,2.59,1572.00,105.71,-1.67,0,1443.05,0.08,-0.79,nan,-0.19,-1.10,-0.01,-0.90,-0.84,-0.05,-0.40
1,100,1,3,2,nan,nan,3898.00,3629.00,27.00,136.02,48.00,3.17,2219.01,93.62,-0.71,0,3738.62,0.79,-0.53,0.47,0.22,-0.55,-0.30,-0.62,0.03,1.29,0.09


In [198]:
dm['gender'].value_counts()

0    226
1    199
Name: gender, dtype: int64

# Table 1. Summary statatisitcs

In [199]:
ds = dm.groupby('sequence')['rsos', 'tsos', 'grip', 'ptiso', 'ntx', 'matu', 'mvh', 'cint'].agg(['count','mean','std']).rename(columns={'count':'N', 'rsos': 'radial', 'tsos': 'tibial', 'grip': 'grip strength', 'ptiso': 'torque', 'ntx': 'Ntx/creatine', 'matu': 'maturity offset', 'mvh': 'mvh', 'cint': 'caloric intake'})
ds.to_csv('./out/t1.csv')
ds

radial               tibial                grip strength             \
              N    mean   std      N    mean    std             N  mean  std   
sequence                                                                       
1            44 3800.25 92.54     44 3641.68 112.28             0   nan  nan   
2            25 3735.68 90.99     35 3653.37  92.76            28 18.88 4.56   
3            62 3787.92 87.98     77 3659.39  98.78            64 20.39 5.84   
4            62 3822.73 95.99     67 3680.94  97.28            65 22.67 6.89   
5            57 3817.28 90.99     71 3683.92 108.66            59 25.01 7.41   
6            70 3858.73 98.75     72 3725.81 103.32            70 24.81 8.04   
7            45 3866.29 93.92     49 3751.51 114.83            43 27.88 8.04   

         torque              Ntx/creatine                 maturity offset  \
              N   mean   std            N    mean     std               N   
sequence                                                                    
1            40 110.18 36.57           42 7527.43 4640.81              44   
2            24  82.16 22.92           35 6202.38 3870.35              38   
3            68 115.37 52.99           76 5650.88 3629.47              77   
4            49 127.84 54.41           61 6004.85 4174.06              71   
5            65 137.75 53.85           69 5867.24 3926.38              71   
6            64 140.80 58.05           69 5881.87 4739.04              74   
7             0    nan   nan           34 5396.64 4007.94              50   

                    mvh              caloric intake                 
          mean  std   N   mean   std              N    mean    std  
sequence                                                            
1        -1.82 1.16  35 119.14 37.14             42 1553.59 531.33  
2        -1.93 1.77  32 109.35 45.91             38 1460.20 413.63  
3        -1.34 1.58  66 121.53 38.44             77 1627.92 473.41  
4        -1.00 2.17  61  90.05 37.14             71 1555.37 457.41  
5        -0.49 1.74  48 101.25 40.73             71 1623.32 480.45  
6        -0.49 2.23  40 112.63 42.79             74 1566.04 480.86  
7         0.38 1.66  25  81.98 39.09             49 1770.87 514.11

Obtain estimation sample

In [200]:
%%stata --data dm --output dm --noprint
set processors 28

xtset id seq

quietly: gsem /// 
    (grips matus ntxs  -> rsoss, ) /// 
    (torqs matus ntxs mvhs -> tsoss, ) /// 
    (cints matus -> grips ,) ///
    (cints matus -> torqs ,) 

gen sall = 1 if e(sample)==1
by id: egen aseq = seq() if sall==1
tab sall

quietly: gsem (grips matus ntxs  -> rsoss, )
gen srsos = 1 if e(sample)==1
by id: egen rseq = seq() if srsos==1

tab srsos

quietly: gsem (torqs matus ntxs mvhs -> tsoss, )
gen stsos = 1 if e(sample)==1
by id: egen tseq = seq() if stsos==1

tab srsos

quietly: gsem (cints matus -> grips ,)
gen sgrip = 1 if e(sample)==1
by id: egen gseq = seq() if sgrip==1
tab sgrip

quietly: gsem (cints matus -> torqs ,) 
gen storq = 1 if e(sample)==1
by id: egen qseq = seq() if storq==1
tab storq

order id session sequence seq aseq rseq tseq gseq qseq


In [201]:
%%stata --data dm 
set processors 28

sum sequence seq aseq rseq tseq gseq qseq


    The maximum number of processors or cores being used is changed from 32 to
    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
    sequence |        425    4.197647    1.833518          1          7
         seq |        425    1.868235    .8776861          1          4
        aseq |        405     1.82716    .8587079          1          4
        rseq |        286    1.562937    .6605875          1          3
        tseq |        220    1.495455    .6374731          1          3
-------------+---------------------------------------------------------
        gseq |        329    1.656535    .7245445          1          3
        qseq |        309    1.621359    .7041137          1          3


In [202]:
de.head()

,id,session,sequence,seq,aseq,rseq,tseq,gseq,qseq,nota,falk,rsos,tsos,grip,ptiso,godin,paq,cint,mvh,matu,gender,ntx,rsoss,tsoss,grips,torqs,ntxs,mvhs,gods,paqs,cints,matus,sall,srsos,stsos,sgrip,storq
0,100,1,1,1,1.00,nan,1.00,nan,1.00,nan,nan,3828.00,3601.00,nan,113.93,37.00,2.59,1572.00,105.71,-1.67,0,1443.05,0.08,-0.79,nan,-0.19,-1.10,-0.01,-0.90,-0.84,-0.05,-0.40,1.00,nan,1.00,nan,1.00
1,100,1,3,2,2.00,1.00,2.00,1.00,2.00,nan,nan,3898.00,3629.00,27.00,136.02,48.00,3.17,2219.01,93.62,-0.71,0,3738.62,0.79,-0.53,0.47,0.22,-0.55,-0.30,-0.62,0.03,1.29,0.09,1.00,1.00,1.00,1.00,1.00
2,100,1,5,3,3.00,nan,3.00,nan,3.00,1.00,nan,nan,3677.00,nan,177.05,44.00,3.21,2599.83,98.14,0.41,0,4027.06,nan,-0.09,nan,0.99,-0.48,-0.19,-0.72,0.10,2.08,0.67,1.00,nan,1.00,nan,1.00
3,100,1,7,4,4.00,2.00,nan,2.00,nan,nan,nan,3952.00,3740.00,40.50,nan,66.00,2.76,2482.12,88.14,1.17,0,2631.34,1.33,0.49,2.27,nan,-0.81,-0.43,-0.17,-0.58,1.83,1.06,1.00,1.00,nan,1.00,nan
4,101,1,1,1,1.00,nan,1.00,nan,1.00,nan,nan,3682.00,3603.00,nan,133.05,25.00,1.71,1549.99,89.29,-1.63,0,9869.88,-1.39,-0.77,nan,0.17,0.93,-0.40,-1.20,-2.16,-0.10,-0.38,1.00,nan,1.00,nan,1.00


In [225]:
d1 = de.groupby('rseq')['rsos'].agg(['count','mean','std']).rename(columns={'count':'n'})
dt = de.groupby(['gender', 'rseq'])['rsos'].agg(['count','mean','std']).rename(columns={'count':'n'})
d1 = d1.append(dt)

d2 = de.groupby('tseq')['tsos'].agg(['count','mean','std']).rename(columns={'count':'n'})
dt = de.groupby(['gender', 'tseq'])['tsos'].agg(['count','mean','std']).rename(columns={'count':'n'})
d2 = d2.append(dt)

d3 = de.groupby('gseq')['grip'].agg(['count','mean','std']).rename(columns={'count':'n'})
dt = de.groupby(['gender', 'gseq'])['grip'].agg(['count','mean','std']).rename(columns={'count':'n'})
d3 = d3.append(dt)

d4 = de.groupby('qseq')['qseq'].agg(['count','mean','std']).rename(columns={'count':'n'})
dt = de.groupby(['gender', 'gseq'])['qseq'].agg(['count','mean','std']).rename(columns={'count':'n'})
d4 = d4.append(dt)

da = pandas.merge(d1, d2, right_index=True, left_index=True, suffixes=('_radial', '_tibial'))
db = pandas.merge(d3, d4, right_index=True, left_index=True, suffixes=('_grip', '_torque'))
d0 = pandas.merge(da, db, right_index=True, left_index=True)

d0.index.name = 'measurement occassion'

# Table 2. Summary data by participant measurement-occasion

In [229]:
d0

,n_radial,mean_radial,std_radial,n_tibial,mean_tibial,std_tibial,n_grip,mean_grip,std_grip,n_torque,mean_torque,std_torque
measurement occassion,,,,,,,,,,,,
1.0,152,3792.14,92.25,128,3645.57,99.08,162,21.27,6.60,157,1.00,0.00
2.0,107,3843.69,89.86,75,3680.57,108.03,118,24.58,7.18,112,2.00,0.00
3.0,27,3876.04,98.62,17,3690.35,92.25,49,27.97,8.51,40,3.00,0.00
"(0, 1.0)",75,3783.87,93.35,61,3643.87,91.95,84,22.38,7.35,58,1.33,0.51
"(0, 2.0)",51,3825.78,82.98,32,3668.44,112.68,59,25.82,8.52,42,2.12,0.63
"(0, 3.0)",13,3875.77,104.88,8,3651.62,80.10,23,30.87,10.73,11,2.36,0.67
"(1, 1.0)",77,3800.19,91.04,67,3647.12,105.83,78,20.08,5.49,66,1.29,0.46
"(1, 2.0)",56,3860.00,93.47,43,3689.60,104.86,59,23.35,5.32,49,2.27,0.53
"(1, 3.0)",14,3876.29,96.42,9,3724.78,92.59,26,25.40,4.80,7,2.14,1.07


Percent changes: all participants

In [230]:
((d0.iloc[2]['mean_radial']/d0.iloc[0]['mean_radial']-1)*100).round(2)
((d0.iloc[2]['mean_tibial']/d0.iloc[6]['mean_tibial']-1)*100).round(2)
((d0.iloc[2]['mean_grip']/d0.iloc[0]['mean_grip']-1)*100).round(2)
((d0.iloc[2]['mean_torque']/d0.iloc[0]['mean_torque']-1)*100).round(2)

2.21

1.1899999999999999

31.48

200.0

Percent changes: boys

In [232]:
((d0.iloc[5]['mean_radial']/d0.iloc[3]['mean_radial']-1)*100).round(2)
((d0.iloc[5]['mean_tibial']/d0.iloc[3]['mean_tibial']-1)*100).round(2)
((d0.iloc[5]['mean_grip']/d0.iloc[3]['mean_grip']-1)*100).round(2)
((d0.iloc[5]['mean_torque']/d0.iloc[3]['mean_torque']-1)*100).round(2)


2.4300000000000002

0.20999999999999999

37.950000000000003

78.040000000000006

Percent change: girls

In [233]:
((d0.iloc[8]['mean_radial']/d0.iloc[6]['mean_radial']-1)*100).round(2)
((d0.iloc[8]['mean_tibial']/d0.iloc[6]['mean_tibial']-1)*100).round(2)
((d0.iloc[8]['mean_grip']/d0.iloc[6]['mean_grip']-1)*100).round(2)
((d0.iloc[8]['mean_torque']/d0.iloc[6]['mean_torque']-1)*100).round(2)

2.0

2.1299999999999999

26.489999999999998

66.390000000000001

# Figure 2. Impact of muscle strength on bone properties

In [ ]:
d0 = dm[dm['gender']==0].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
d1 = dm[dm['gender']==1].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()

t1 = ds.append(d0)
t1 = t1.append(d1)
#t1.to_csv("./out/t1.csv")
t1

In [58]:
%%stata --data dm --output dm
set processors 20

xtset id seq
gsem /// 
    (M1[id]  grips matus ntxs  -> rsoss, ) /// 
    (N1[id] cints matus -> grips ,)  ///
    (L1[id] -> ntxs) ///
    , cov(M1[id]*N1[id])


    The maximum number of processors or cores being used is changed from 32 to       panel variable:  id (unbalanced)
        time variable:  seq, 1 to 4
                delta:  1 unit
>     (M1[id]  grips matus ntxs  -> rsoss, ) /// 
>     (N1[id] cints matus -> grips ,)  ///
>     (L1[id] -> ntxs) ///
>     , cov(M1[id]*N1[id])

Fitting fixed-effects model:

Iteration 0:   log likelihood = -1255.4009  
Iteration 1:   log likelihood = -1255.4009  

Refining starting values:

Grid node 0:   log likelihood = -1297.9248

Fitting full model:

Iteration 0:   log likelihood = -1297.9248  (not concave)
Iteration 1:   log likelihood = -1199.8679  (not concave)
Iteration 2:   log likelihood = -1177.7375  (not concave)
Iteration 3:   log likelihood =   -1130.34  
Iteration 4:   log likelihood = -1112.1945  
Iteration 5:   log likelihood = -1108.3927  
Iteration 6:   log likelihood = -1108.2861  
Iteration 7:   log likelihood = -1108.2852  
Iteration 8:   log likelihood = -1108.2852  

Generali

# Figure 3. Constrained specification results.

In [114]:
%%stata --data dm 
set processors 28

xtset id sequence

gsem /// 
    (M1[id]  grips@a matus@b ntxs@c  -> rsoss, ) /// 
    (M1[id]  torqs@a matus@b ntxs@c mvhs -> tsoss, ) /// 
    (N1[id] cints@n matus@m -> grips  ,) ///
    (N1[id] cints@n matus@m -> torqs ,) ///
    , cov(e.rsoss*e.tsoss) cov(e.grips*e.torqs) 

*******
count


    The maximum number of processors or cores being used is changed from 32 to       panel variable:  id (unbalanced)
        time variable:  sequence, 1 to 7, but with gaps
                delta:  1 unit
>     (M1[id]  grips@a matus@b ntxs@c  -> rsoss, ) /// 
>     (M1[id]  torqs@a matus@b ntxs@c mvhs -> tsoss, ) /// 
>     (N1[id] cints@n matus@m -> grips  ,) ///
>     (N1[id] cints@n matus@m -> torqs ,) ///
>     , cov(e.rsoss*e.tsoss) cov(e.grips*e.torqs) cov(M1[id]*N1[id]@0) 

Fitting fixed-effects model:

Iteration 0:   log likelihood = -1329.2829  
Iteration 1:   log likelihood = -1296.7212  (backed up)
Iteration 2:   log likelihood = -1276.0733  
Iteration 3:   log likelihood = -1258.1107  
Iteration 4:   log likelihood =   -1257.32  
Iteration 5:   log likelihood = -1257.3194  
Iteration 6:   log likelihood = -1257.3194  

Refining starting values:

Grid node 0:   log likelihood = -1317.5362

Fitting full model:

Iteration 0:   log likelihood = -1317.5362  (not concave)
Itera

In [111]:
len(dm.groupby('id'))
len(dm[dm.gender==0].groupby('id'))
len(dm[dm.gender==1].groupby('id'))

81

In [192]:
dm.groupby(['session'])['sequence'].value_counts()

session  sequence
1        3           75
         5           70
         7           43
         1           39
2        6           73
         4           70
         2           35
Name: sequence, dtype: int64